In [2]:
from datetime import datetime, timedelta
import json
import uuid
import random
import os
from sqlalchemy import create_engine

from utils import get_session, model_to_dict
from data.models import cultpass

# Udahub Accounts

## Cultpass Database

**Init DB**

In [3]:
cultpass_db = "data/external/cultpass.db"

In [4]:
import os

# Remove and recreate the database with cultpass schema
if os.path.exists(cultpass_db):
    os.remove(cultpass_db)
    print(f"✅ Removed existing {cultpass_db}")

# Create engine and tables with cultpass schema
engine = create_engine(f"sqlite:///{cultpass_db}", echo=True)
cultpass.Base.metadata.create_all(engine)

table_names = ", ".join(cultpass.Base.metadata.tables.keys())
print(f"✅ Recreated {cultpass_db} with cultpass schema")
print(f"   Tables created: {table_names}")

✅ Removed existing data/external/cultpass.db
2025-11-05 23:24:54,584 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-05 23:24:54,584 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-11-05 23:24:54,584 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-05 23:24:54,584 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-11-05 23:24:54,585 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-05 23:24:54,585 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("subscriptions")
2025-11-05 23:24:54,585 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-05 23:24:54,586 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("subscriptions")
2025-11-05 23:24:54,586 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-05 23:24:54,586 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("experiences")
2025-11-05 23:24:54,586 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-05 23:24:54,586 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("experiences")
2025-

In [5]:
# Engine is already created in cell 3, but we redefine it here for clarity
engine = create_engine(f"sqlite:///{cultpass_db}", echo=False)

**Experiences**

In [6]:
experience_data = []

with open('data/external/cultpass_experiences.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        experience_data.append(json.loads(line))

In [7]:
experience_data

[{'title': 'Carnival History Tour in Olinda',
  'description': "Discover the origins and vibrant traditions of Pernambuco's Carnival.",
  'location': 'Pernambuco, Brazil'},
 {'title': 'Sunset Paddleboarding',
  'description': 'Glide across calm waters at golden hour with all gear included.',
  'location': 'Santa Catarina, Brazil'},
 {'title': 'Pelourinho Colonial Walk',
  'description': 'Wander through colorful streets and learn about Afro-Brazilian history.',
  'location': 'Bahia, Brazil'},
 {'title': 'Samba Night at Lapa',
  'description': 'Dance the night away at a traditional samba club in the Lapa arches.',
  'location': 'Rio de Janeiro, Brazil'},
 {'title': 'Christ the Redeemer Experience',
  'description': 'Take a guided trip to one of the New Seven Wonders of the World with historical context.',
  'location': 'Rio de Janeiro, Brazil'},
 {'title': 'Modern Art at MASP',
  'description': 'Enjoy a guided visit to the São Paulo Museum of Art with insights into its top collections.',

In [8]:
with get_session(engine) as session:
    experiences = []

    for idx, experience in enumerate(experience_data):
        exp = cultpass.Experience(
            experience_id=str(uuid.uuid4())[:6],
            title=experience["title"],
            description=experience["description"],
            location=experience["location"],
            when=datetime.now() + timedelta(days=idx+1),
            slots_available=random.randint(1,30),
            is_premium=(idx % 2 == 0)
        )
        experiences.append(exp)

    session.add_all(experiences)

**User**

In [9]:
cultpass_users = []

with open('data/external/cultpass_users.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        cultpass_users.append(json.loads(line))

In [10]:
cultpass_users

[{'id': 'a4ab87',
  'name': 'Alice Kingsley',
  'email': 'alice.kingsley@wonderland.com',
  'is_blocked': True},
 {'id': 'f556c0',
  'name': 'Bob Stone',
  'email': 'bob.stone@granite.com',
  'is_blocked': False},
 {'id': '88382b',
  'name': 'Cathy Bloom',
  'email': 'cathy.bloom@florals.org',
  'is_blocked': False},
 {'id': '888fb2',
  'name': 'David Noir',
  'email': 'david.noir@shadowmail.com',
  'is_blocked': True},
 {'id': 'f1f10d',
  'name': 'Eva Green',
  'email': 'eva.green@ecosoul.net',
  'is_blocked': False},
 {'id': 'e6376d',
  'name': 'Frank Ocean',
  'email': 'frank.ocean@seawaves.io',
  'is_blocked': False}]

In [11]:
with get_session(engine) as session:
    db_users = []
    for user_info in cultpass_users:
        user = cultpass.User(
            user_id=user_info["id"],
            full_name=user_info["name"],
            email=user_info["email"],
            is_blocked=user_info["is_blocked"],
            created_at=datetime.now()
        )
        db_users.append(user)
    session.add_all(db_users) 

**Subscription**

In [12]:
with get_session(engine) as session:
    subscriptions = []
    for user_info in cultpass_users:
        subscription = cultpass.Subscription(
            subscription_id=str(uuid.uuid4())[:6],
            user_id=user_info["id"],
            status=random.choice(["active", "cancelled"]),
            tier=random.choice(["basic", "premium"]),
            monthly_quota=random.randint(2,10),
            started_at=datetime.now()
        )
        subscriptions.append(subscription)

    session.add_all(subscriptions)

**Reservation**

In [13]:
# Applicable to `cultpass_users[0]` at the moment

with get_session(engine) as session:
    experience_ids = [
        exp.experience_id 
        for exp 
        in session.query(cultpass.Experience).all()
    ]

    reservation1 = cultpass.Reservation(
        reservation_id=str(uuid.uuid4())[:6],
        user_id=cultpass_users[0]["id"],
        experience_id=random.choice(experience_ids),
        status="reserved",
    )

    reservation2 = cultpass.Reservation(
        reservation_id=str(uuid.uuid4())[:6],
        user_id=cultpass_users[0]["id"],
        experience_id=random.choice(experience_ids),
        status="reserved",
    )

    session.add_all([reservation1, reservation2])

In [14]:
# TODO: Add more data
# Please notice that the reservations were set to first user only 
# If you want to simulate more users later, please create more reservations per user

with get_session(engine) as session:
    pass

# Tests

In [15]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user)

<User(user_id='a4ab87', email='alice.kingsley@wonderland.com', is_blocked=True)>
<User(user_id='f556c0', email='bob.stone@granite.com', is_blocked=False)>
<User(user_id='88382b', email='cathy.bloom@florals.org', is_blocked=False)>
<User(user_id='888fb2', email='david.noir@shadowmail.com', is_blocked=True)>
<User(user_id='f1f10d', email='eva.green@ecosoul.net', is_blocked=False)>
<User(user_id='e6376d', email='frank.ocean@seawaves.io', is_blocked=False)>


In [16]:
with get_session(engine) as session:
    users = session.query(cultpass.User).all()
    for user in users:
        print(user.subscription)

<Subscription(subscription_id='f7020f', user_id='a4ab87', status='active', tier='premium')>
<Subscription(subscription_id='b583a7', user_id='f556c0', status='active', tier='premium')>
<Subscription(subscription_id='52020d', user_id='88382b', status='active', tier='premium')>
<Subscription(subscription_id='9ffda3', user_id='888fb2', status='cancelled', tier='premium')>
<Subscription(subscription_id='ff968b', user_id='f1f10d', status='active', tier='basic')>
<Subscription(subscription_id='ef9b49', user_id='e6376d', status='active', tier='basic')>


In [17]:
with get_session(engine) as session:
    experiences = session.query(cultpass.Experience).all()
    for experience in experiences:
        print(experience)

<Experience(experience_id='ec4ecc', title='Carnival History Tour in Olinda', when='2025-11-06 23:24:54.607001')>
<Experience(experience_id='3ce895', title='Sunset Paddleboarding', when='2025-11-07 23:24:54.609971')>
<Experience(experience_id='883360', title='Pelourinho Colonial Walk', when='2025-11-08 23:24:54.609995')>
<Experience(experience_id='c96c90', title='Samba Night at Lapa', when='2025-11-09 23:24:54.610012')>
<Experience(experience_id='ed07b2', title='Christ the Redeemer Experience', when='2025-11-10 23:24:54.610025')>
<Experience(experience_id='113f41', title='Modern Art at MASP', when='2025-11-11 23:24:54.610040')>
<Experience(experience_id='bd606f', title='Ibirapuera Park Bike Ride', when='2025-11-12 23:24:54.610051')>
